# **Multi-Compartment Model**
*This script apllies a multi-compartment model designed for gery-matter morphology to multishell DWI data over selected segmented subcortical structures and saves all fitted parameters into a dataframe that is saved as a json file. For some reason the multi-compartment model is reluctant to work when put into a loop looping over all subjects and for that reason the script only processes one subject, storing all parameters for one subcortical structure into a cell in a dataframe. This results in a dataframe with one row (will later be merged together).*

In [ ]:
# import packages
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from dmipy.core.acquisition_scheme import acquisition_scheme_from_bvalues
from dmipy.signal_models import cylinder_models, sphere_models, gaussian_models
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models import cylinder_models, gaussian_models, sphere_models
from dmipy.distributions import distribute_models
import pandas as pd
from scipy.ndimage import binary_erosion

### **Defining Functions**
*A few functions for necessary tasks were pre-defined for improved readability of code*

- *Function for loading a nifti file (load_nifti)*
    - *This function takes a filename as an argument, loads a nifti file by this name, extracts the data matrix and returns it*
- *Function for extracting the mask of one structure from the mask file (extract_single_mask)*
    - *This function takes two arguments. The first argument is the mask file, containing the segmented mask of every segmented structure. The second argument, the "mask_index" is an array containing two values corresponding to the labels of the left and right of the desired subcortical structure (i.e. left and right amygdala). The function proceeds to copy the original mask file (as to not interfer with the original data), collect the correct structures, make the mask binary and perfom erosion before returning the eroded binary mask.*
- *Function for extracting all values from the DWI file belonging to mask (extract_values)*
    - *This function takes two arguments, the full DWI data matrix and a mask (result from extract_single_mask). The function returns a 3-dimensional that only contains the voxels covered by the maks*

In [ ]:
def load_nifti(filename):
    nifti_file = nib.load(filename)
    data = nifti_file.get_fdata()
    return data

def extract_single_mask(all_masks_file, mask_index):
    mask = all_masks_file.copy()
    mask[(mask != mask_index[0]) & (mask != mask_index[1])]  = 0  #all values not the right label to 0
    mask = (mask != 0).astype(np.uint8) #make it binary
    mask = binary_erosion(mask, iterations=1).astype(mask.dtype) # perform erosion to get rid of edge values
    return mask

def extract_values(dwi_data, mask):
    nonzero = np.where(mask > 0)
    values = dwi_data[nonzero] 
    return values

### **Set up acquisition scheme**
*Seperate files containing the b-values, b-vectors and small delta ($\delta$) and big delta ($\Delta$) values used during acquisition is loaded to be used for setting up the acquisition scheme. All values in these files are ordered the same as the DWI shells in the merged data file and after loading they are converted to SI units*

In [ ]:
# set up acquisition scheme
bval = np.loadtxt('all.bval') * 1e6 # load and convert from [s/mm²] to SI unit [s/m²]
bvec = np.transpose(np.loadtxt('all.bvec'))
delta = np.loadtxt('small_delta.txt')*10e-4 #load and convert from [ms] to SI unit [s] 
Delta = np.loadtxt('big_delta.txt')*10e-4 #load and convert from [ms] to SI unit [s] 

acq_scheme = acquisition_scheme_from_bvalues(bval, bvec, delta, Delta)

### **Initialize the multi-compartment model**
*In the following cell the multi-compartemnt model used is initialized by defining the necesary compartments and apllying the desired constraints. In short the model makes use of:*
- *Two spherical compartments off restricted diffusion designed to model the microglia and astrocytes. As restricted diffusion is assumed to be 1.0e-09 m²/s, the diffusion constant is set to this value. Later ranges for the diameters of the spheres are also defined with 1 - 10 mm for the small sphere and 10-30 for the large sphere.*
- *One ball compartment to represent freely diffusing water. The diffusivity of free water is usually assumed to be 3.0e-09 m²/s and the $\lambda_{iso}$ parameter is therefore set to this value.*
- *A stick and a zeppelin combined in a watson dispersed bundle to represent diffusion in microglial processes and axons and the extracellular space. The diffusivity within restriction (diffusivity of the stick) is still assumed to be 1.0e-09 m²/s.*

In [ ]:
small_sphere = sphere_models.S4SphereGaussianPhaseApproximation(diffusion_constant=1.0e-9) # diffusivity within restriction set to 1.0e-09 mm²/s
big_sphere = sphere_models.S4SphereGaussianPhaseApproximation(diffusion_constant=1.0e-9) # diffusivity within restriction set to 1.0e-09 mm²/s
free_water = gaussian_models.G1Ball()
stick = cylinder_models.C1Stick()
zeppelin = gaussian_models.G2Zeppelin()
watson_dispersed_bundle = distribute_models.SD1WatsonDistributed(models=[stick, zeppelin])   
watson_dispersed_bundle.set_fixed_parameter('C1Stick_1_lambda_par', 1.0e-9) # diffusivity within restriction set to 1.0e-09 mm²/s

microglia_model = MultiCompartmentModel(models=(small_sphere, big_sphere, free_water, watson_dispersed_bundle))

microglia_model.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_1_diameter',[1.0e-6, 1.0e-5])# small sphere diameter range
microglia_model.set_parameter_optimization_bounds('S4SphereGaussianPhaseApproximation_2_diameter',[1.0e-5, 3.0e-5])# big sphere diameter range 
microglia_model.set_fixed_parameter('G1Ball_1_lambda_iso', 3.0e-09) # diffusivity of free water set to 1.0e-09 mm²/s

### **Initialize Dataframe**
*For practical storing of fitted parameters a dataframe was created. The dataframe uses the subject ID as the row index and the names of the subcortical structures as column names. After the model is fitted to the data the each cell will be filled with a dictionary containing all fitted values of every parameter for the structure of that column. Below a dictionary named "subcortical_structures_dict" is also defined. It is used both for initializing the dataframe, but also for processing the data later as this dict contains the labels of the different subcortical structures in the mask needed for extracting one structure from the original file*

In [ ]:

ID = ''  # Choose subject

subcortical_structures_dict = {
    'Thalamus': [10, 49],
    'Caudate':[11, 50],
    'Putamen': [12, 51],
    'Pallidum': [13, 52],
    'Hippocampus': [17, 53],
    'Amygdala': [18, 54],
    'Accumbens Area': [26, 58] }  

df = pd.DataFrame(index=[ID] , columns=[i for i in subcortical_structures_dict])


### **Applying model to DWI data**
*The cell below apllies the model to the DWI data. It is structured so that the DWI data and the mask for one subject is loaded. Then a loop extracts the voxels of one structure at a time, apllying the model to these voxels and saving the fitted parameters to the dataframe.*


In [ ]:
file = ID + '.nii.gz'
dwi_data = load_nifti('pre_processing/4_normalized_for_TE/' + file)
mask_data = load_nifti('pre_processing/5_segmentation/masks/' + file)

for structure in subcortical_structures_dict:
    print()
    print('Structure: ', structure)
    print('structure indexes: ', subcortical_structures_dict[structure])
    structure_mask = extract_single_mask(mask_data, subcortical_structures_dict[structure])
    structure_data_flattened = extract_values(dwi_data, structure_mask)
    print('Voxels covered by mask: ', len(structure_data_flattened))

    fitted_model = microglia_model.fit(acq_scheme, structure_data_flattened, solver='mix')
    df.loc[ID, structure] = [fitted_model.fitted_parameters]

In [ ]:
# Save dataframe as json file
df.to_json('Fitted_parameters_dataframes/Fitted_parameters_'+ID+'.json', orient='index')